In [22]:
import os
import librosa
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [23]:
# Read csv file 
# Data was downloaded from https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
#There are audio files and csv file. For this part csv with 30 sec music features was used.
path = "csv\mldata_v2.csv"
df = pd.read_csv(path)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav_1,66150,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829707,1972.334258,117272.640573,...,39.547077,-3.230046,36.606857,0.696385,37.766136,-5.035945,33.668550,-0.239585,43.818886,blues
1,blues.00000.wav_2,66150,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297514,2010.751494,65940.666037,...,64.819780,-6.025473,40.548813,0.127131,51.048943,-2.808956,97.221504,5.771881,60.360348,blues
2,blues.00000.wav_3,66150,0.347746,0.092495,0.130895,0.004552,1790.722358,110071.206762,2088.184750,73391.498088,...,68.306790,-1.714475,28.136944,2.329553,47.211426,-1.925621,52.922430,2.466996,33.163998,blues
3,blues.00000.wav_4,66150,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936309,1967.920582,79805.901501,...,48.543200,-3.786986,28.419546,1.153315,35.682700,-3.501979,50.610344,3.580636,32.325880,blues
4,blues.00000.wav_5,66150,0.335481,0.088482,0.142370,0.001734,1634.465076,77425.419156,1954.633566,57359.695597,...,30.829544,0.635798,44.645557,1.591107,51.415867,-3.364908,26.421090,0.501504,29.109533,blues


In [24]:
df.columns

Index(['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var',
       'spectral_flatness_mean', 'spectral_flatness_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var',
       'tonnetz_mean', 'tonnetz_var', 'chroma_cens_mean', 'chroma_cens_var',
       'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var',
       'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean',
       'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var',
       'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean',
       'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var',
       'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean',
       'mfcc15_var', 'mfcc16_mean', 'mfc

In [25]:
df.shape[1]

66

In [26]:
# Prepare features (X) and target (y)
df.drop(["filename","length"], axis=1, inplace=True)  # Drop unnecessary columns
X = df.drop("label", axis=1)  # Features
y = df["label"]  # Target label

In [27]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [28]:
# One-Hot Encoding
y_categorical = to_categorical(y_encoded)
y_categorical

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [29]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [30]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y_categorical,random_state=1)

#1 Neural Network Model

In [31]:
# Create a Sequential model
model = Sequential()

model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

c:\Users\weiwei\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,858 (73.66 KB)

 Trainable params: 18,858 (73.66 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# train the NN model (# early_stop was recommended by ChatGPT to improve the accuracy of model training.)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stop])



Epoch 1/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1687 - loss: 0.4558 - val_accuracy: 0.4823 - val_loss: 0.2354
Epoch 2/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3913 - loss: 0.2713 - val_accuracy: 0.6012 - val_loss: 0.1958
Epoch 3/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5021 - loss: 0.2328 - val_accuracy: 0.6480 - val_loss: 0.1716
Epoch 4/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5498 - loss: 0.2092 - val_accuracy: 0.6914 - val_loss: 0.1565
Epoch 5/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6176 - loss: 0.1878 - val_accuracy: 0.7128 - val_loss: 0.1467
Epoch 6/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6320 - loss: 0.1789 - val_accuracy: 0.7201 - val_loss: 0.1406
Epoch 7/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6515 - loss: 0.1743 - val_accuracy: 0.7341 - val_loss: 0.1333
Epoch 8/100
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6709 - loss: 0.1647 - val_accu

In [33]:
# Evaluate the NN model
test_loss, test_accuracy = model.evaluate(X_test, y_test,verbose=1)
print(test_accuracy)

78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8942 - loss: 0.0573
0.8862179517745972


In [34]:
# Predict class probabilities for the test set
y_pred = model.predict(X_test)

# Convert predictions from probabilities to class indices
y_pred_classes = y_pred.argmax(axis=1)

# Convert one-hot encoded y_test back to class indices
y_test_classes = y_test.argmax(axis=1)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [35]:
# Classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))


# Source: The Multi-Class classification approach was suggested by ChatGPT to effectively classify all genres and improve 
# accuracy. Instead of using binary classification, a softmax activation function with categorical cross-entropy loss was 
# implemented to handle multiple classes, leading to better model performance.

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       258
           1       0.95      0.97      0.96       242
           2       0.88      0.87      0.88       263
           3       0.88      0.78      0.83       234
           4       0.92      0.82      0.87       242
           5       0.87      0.94      0.90       248
           6       0.94      0.96      0.95       264
           7       0.90      0.89      0.89       257
           8       0.83      0.87      0.85       245
           9       0.78      0.81      0.79       243

    accuracy                           0.89      2496
   macro avg       0.89      0.88      0.88      2496
weighted avg       0.89      0.89      0.89      2496



#2  XGBoost Model

In [36]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, random_state=1)

In [37]:
# Create the XGBoost model
xgb_model = xgb.XGBClassifier(n_estimators=100)

In [38]:
# Train the XGBoost model
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [39]:
# Make predictions
y_pred = xgb_model.predict(X_test)


In [40]:
# Evaluate the model
test_accuracy = np.mean(y_pred == y_test)
test_accuracy


0.9094551282051282

In [41]:
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Source: XGboost was recommedated by our TA Andrew and ChatGPT(OpenAI, January 2025).
# Additional Reference: IBM. "XGBoost - A Powerful Machine Learning Algorithm." IBM Think.

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93       258
           1       0.95      0.97      0.96       242
           2       0.87      0.88      0.87       263
           3       0.91      0.88      0.90       234
           4       0.93      0.90      0.92       242
           5       0.91      0.92      0.92       248
           6       0.95      0.95      0.95       264
           7       0.92      0.89      0.91       257
           8       0.91      0.89      0.90       245
           9       0.84      0.86      0.85       243

    accuracy                           0.91      2496
   macro avg       0.91      0.91      0.91      2496
weighted avg       0.91      0.91      0.91      2496



#3 Catboost

In [44]:
pip install catboost


   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   - -------------------------------------- 4.7/101.7 MB 26.1 MB/s eta 0:00:04
   ---- ----------------------------------- 11.5/101.7 MB 30.1 MB/s eta 0:00:04
   ------- -------------------------------- 18.4/101.7 MB 29.7 MB/s eta 0:00:03
   ---------- ----------------------------- 25.7/101.7 MB 31.3 MB/s eta 0:00:03
   ------------ --------------------------- 32.0/101.7 MB 30.8 MB/s eta 0:00:03
   --------------- ------------------------ 39.3/101.7 MB 31.2 MB/s eta 0:00:02
   ------------------ --------------------- 47.2/101.7 MB 32.0 MB/s eta 0:00:02
   --------------------- ------------------ 53.5/101.7 MB 31.8 MB/s eta 0:00:02
   ----------------------- ---------------- 60.3/101.7 MB 31.8 MB/s eta 0:00:02
   -------------------------- ------------- 67.9/101.7 MB 32.1 MB/s eta 0:00:02
   ----------------------------- ---------- 74.4/101.7 MB 32.1 MB/s eta 0:00:01
   -------------------------------- ------- 82.3/

In [45]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [47]:
# Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(iterations=1500,
                                    depth=6,             
                                    learning_rate=0.1,  
                                    loss_function='MultiClass',   
                                    verbose=50)       


In [48]:
# Train the model
catboost_model.fit(X_train, y_train)

0:	learn: 2.1533009	total: 298ms	remaining: 7m 27s
50:	learn: 0.8258424	total: 7.53s	remaining: 3m 33s
100:	learn: 0.5777068	total: 14.5s	remaining: 3m 21s
150:	learn: 0.4391085	total: 21.6s	remaining: 3m 13s
200:	learn: 0.3544204	total: 28.9s	remaining: 3m 6s
250:	learn: 0.2990544	total: 36.7s	remaining: 3m 2s
300:	learn: 0.2553272	total: 44.1s	remaining: 2m 55s
350:	learn: 0.2243533	total: 51.3s	remaining: 2m 47s
400:	learn: 0.1980880	total: 58.6s	remaining: 2m 40s
450:	learn: 0.1740424	total: 1m 5s	remaining: 2m 33s
500:	learn: 0.1548284	total: 1m 13s	remaining: 2m 25s
550:	learn: 0.1394278	total: 1m 20s	remaining: 2m 18s
600:	learn: 0.1261747	total: 1m 27s	remaining: 2m 10s
650:	learn: 0.1151409	total: 1m 34s	remaining: 2m 3s
700:	learn: 0.1049494	total: 1m 41s	remaining: 1m 56s
750:	learn: 0.0951711	total: 1m 49s	remaining: 1m 49s
800:	learn: 0.0878722	total: 1m 56s	remaining: 1m 41s
850:	learn: 0.0803603	total: 2m 4s	remaining: 1m 34s
900:	learn: 0.0742127	total: 2m 12s	remaining

In [49]:
# Evaluate the model
y_pred = catboost_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {test_accuracy}')
print(classification_report(y_test, y_pred))

# Source: CatBoost was recommended by ChatGPT, and we found it achieved the highest accuracy during model training.
# CatBoost is a powerful gradient boosting algorithm known for handling categorical features efficiently. 
# After comparing various models, we chose CatBoost for its strong performance and ease of use.
# code reference:https://github.com/catboost/catboost

Test Accuracy: 0.9274839743589743
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       258
           1       0.96      0.98      0.97       242
           2       0.90      0.91      0.90       263
           3       0.92      0.89      0.91       234
           4       0.95      0.93      0.94       242
           5       0.89      0.96      0.93       248
           6       0.96      0.97      0.96       264
           7       0.94      0.91      0.92       257
           8       0.91      0.90      0.91       245
           9       0.90      0.87      0.88       243

    accuracy                           0.93      2496
   macro avg       0.93      0.93      0.93      2496
weighted avg       0.93      0.93      0.93      2496

